In [30]:
import numpy as np
import librosa
import os
from tqdm import tqdm
from scipy import signal
import warnings

warnings.filterwarnings('ignore')

def featureCal(y,sr,times_len):
    chroma_stft = librosa.feature.chroma_stft(y,sr)
    spectral_center = librosa.feature.spectral_centroid(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y,sr,n_mfcc=20)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
#     print(chroma_stft.shape,spectral_center.shape,mfcc.shape,spectral_contrast.shape)
    features = chroma_stft.copy()
    for feature in [spectral_center,mfcc,spectral_contrast]:
        features = np.append(features,feature,axis=0)
    features = signal.resample(features,times_len,axis=1)
    return features[:,:times_len]
genres = os.listdir('../input/gtzan-genre-collection/genres/')
genres.sort()
print(genres)

features = []
labels = []
for i,genre in enumerate(genres):
    files = os.listdir(f'../input/gtzan-genre-collection/genres/{genre}/')
    files.sort()
    for file in tqdm(files):
        y,sr = librosa.load(f"../input/gtzan-genre-collection/genres/{genre}/{file}")
        feature = featureCal(y,sr,times_len=256)
        features.append(feature)
        labels.append(i)
features = np.stack(features)
labels = np.array(labels)
# np.save(f"{genre}_features.npy",features)
# np.save(f"{genre}_labels.npy",labels)
# features = np.stack(features)
np.save(f"features.npy",features)
np.save(f"labels.npy",labels)

In [34]:
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder, StandardScaler

# features = np.load("../input/cnnmusic/features.npy")
# labels = np.load("../input/cnnmusic/labels.npy")
features = np.load("./features.npy")
labels = np.load("./labels.npy")
features = np.transpose(features,[0,2,1])
scaler = StandardScaler()
for i in range(features.shape[0]):
    features[i,:,:] = scaler.fit_transform(features[i,:,:])
print(features.shape)
x_train,x_test,y_train,y_test=train_test_split(features,labels,test_size=0.3,random_state=0)   #划分验证集
print(x_train.shape)
print(y_train.shape)

In [64]:
from keras import models
from keras import layers

# input_shape = (666, 40, 1201)
model = models.Sequential()
model.add(layers.Conv1D(64,3,activation='relu',input_shape=x_train.shape[1:]))
# model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(32,3,activation='relu'))
model.add(layers.Conv1D(16,3,activation='relu'))
# model.add(layers.Dropout(0.1))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

history = model.fit(x_train,y_train,epochs=40,batch_size=32)
test_loss, test_acc = model.evaluate(x_test,y_test)
print('test_acc: ',test_acc)
predictions = model.predict(x_test)
model.save('Audino_CNN')

In [65]:
classlist = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
predixt = np.argmax(predictions,axis=1)
# for i, j in zip(y_test[:20], predixt[:20]):
#     print(f"Predict:{classlist[int(j)]:<10}Real:{classlist[int(i)]:<10}Status:{i==j}")

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,predixt)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=[10,10])
sns.heatmap(matrix,cmap='Blues',annot=True,fmt='.20g',xticklabels=classlist,yticklabels=classlist)
plt.xlabel('Predict')
plt.ylabel('Real')

np.save('matrix',matrix)